### Demo tempo change with beethoven sonata

In [2]:
import mido
import sys
import binascii
import pprint
from collections import OrderedDict
from mido import Message, MidiFile, MidiTrack
from enum import Enum
from copy import copy, deepcopy

In [3]:
### Need to deal with variable length -- for both "len" of meta-event, and "delta-time"
### Need to deal with running status -- http://midi.teragonaudio.com/tech/midispec/run.htm

MSB_1 = ['8', '9', 'a', 'b', 'c', 'd', 'e', 'f']

## TODO: ff00
## TODO: ff20, 21
## TODO: ff7f
## TODO: running status for all events

END_OF_TRACK = 'ff2f00'
TEXT_META_EVENTS = [f'ff0{i+1}' for i in range(9)]
TEMPO = 'ff5103'
SMPTE = 'ff5405'
TIME_SIG = 'ff5804'
KEY_SIG = 'ff5902'

class Track:
    def __init__(self, track):
        self.track = track
        self._parse()
    
    def _get_varlen_value (self, string):
        value = ''
        j = 0
        while True:
            if string[j] not in MSB_1: # reached last byte of variable-length field
                value = string[0:j+2]
                break
            else:
                j += 2
        return value
        
    def _varFieldToInt(self, hexstring):
        ret = 0
        current_byte = ''

        i=0
        while len(hexstring):
            current_byte = "0x" + hexstring[0:2]
            current_integer = int(current_byte, 16)
            ret = (ret << 7) | (current_integer & 0x7f)
            if not current_integer & 0x80:
                return ret
            hexstring = hexstring[2:]

            i += 1
            if i>1000:
                print('error')
                break
    
    def _parse(self):
        D = []
        
        assert self.track[0:8] == '4d54726b'
        self.chunklen = self.track[8:8+8]
        D.append({'type':'header', 'value':'4d54726b'})
        D.append({'type':'chunklen', 'value':self.track[8:8+8]})
        
        track = self.track[16:]
        i = 0
        state = 'delta-time' # state in ['delta-time', 'event']
        while True:
            if state == 'delta-time':
                value = self._get_varlen_value(track[i:])
                D.append( {'type':'delta time', 'value':value} )
                i += len(value)
                state = 'event'
                continue
                
            else: # state == 'event'
                running = False
                ## first decode event
                if track[i] not in MSB_1: # running status in use
                    event = last_event # for note on / note off / control change / program change - include channel n
                    running = True
                else:
                    track_first6 = track[i:i+6]
                    track_first4 = track[i:i+4]
                    track_first = track[i]
                    if track_first6 == END_OF_TRACK: # end of track; ff 2f 00
                        event = END_OF_TRACK
                    elif track_first6 == TIME_SIG: # ff 58 04 nn dd cc bb
                        event = TIME_SIG
                    elif track_first6 == KEY_SIG: # ff 59 02 sf mi
                        event = KEY_SIG
                    elif track_first6 == TEMPO: # ff 51 03 tt tt tt
                        event = TEMPO
                    elif track_first6 == SMPTE: # ff 54 05 hr mn se fr ff
                        event = SMPTE
                    elif track_first4 in TEXT_META_EVENTS:
                        event = track_first4
                    elif track_first in ['b','c','8','9']:
                        event = track[i:i+2]
                    
                ## then parse event and append to event list
                if event == END_OF_TRACK: # end of track; ff 2f 00
                    D.append( {'type':'end of track', 'type hex':END_OF_TRACK, 'value':''} )
                    break
                
                elif event == TIME_SIG: # ff 58 04 nn dd cc bb
                    D.append( {'type':'time signature', 'type hex':track_first6, 'value':track[i+6:i+6+8]} )
                    i += 14
                    
                elif event == KEY_SIG: # ff 59 02 sf mi
                    D.append( {'type':'key signature', 'type hex':track_first6, 'value':track[i+6:i+6+4]} )
                    i += 10
                
                elif event == TEMPO: # ff 51 03 tt tt tt
                    D.append( {'type':'tempo', 'type hex':track_first6, 'value':track[i+6:i+6+6]} )
                    i += 12
                
                elif event == SMPTE: # ff 54 05 hr mn se fr ff
                    D.append( {'type':'smpte', 'type hex':track_first6, 'value':track[i+6:i+6+10]} )
                    i += 16
                    
                # sequence/track name (ff 03 length text)
                # or marker (ff 06 length text)
                # or cue (ff 07 length text)
                elif event in TEXT_META_EVENTS:
                    if running:
                        name_length_varlen = self._get_varlen_value(track)
                        name_length = self._varFieldToInt(name_length_varlen)
                        name = track[i + len(name_length_varlen) : i + len(name_length_varlen) + name_length*2]
                        D.append( {'type':'text meta event', 'type hex':event, 'omitted':2, ## omitted 2 bytes
                                   'length_varlen':name_length_varlen, 'name':name} )
                        i += len(name_length_varlen) + name_length*2 # name_length * byte, every byte == 2 hex
                    else:
                        name_length_varlen = self._get_varlen_value(track[i+4:])
                        name_length = self._varFieldToInt(name_length_varlen)
                        name = track[i + 4 + len(name_length_varlen) : i + 4 + len(name_length_varlen) + name_length*2]
                        D.append( {'type':'text meta event', 'type hex':track_first4, 'omitted':0,
                                   'length_varlen':name_length_varlen, 'name':name} )
                        i += 4 + len(name_length_varlen) + name_length*2 # name_length * byte, every byte == 2 hex
                
                # Assuming only bn/cn/8n/9n can use running-status
                elif event[0] == 'b': # control change; bn controller value
                    if running:
                        D.append( {'type':'control change', 'type hex':'b', 'omitted':1,
                                   'channel':event[1], 'controller':track[i:i+2], 'value':track[i+2:i+4]} )
                        i += 4
                    else:
                        D.append( {'type':'control change', 'type hex':'b', 'omitted':0,
                                   'channel':track[i+1], 'controller':track[i+2:i+4], 'value':track[i+4:i+6]} )
                        i += 6
                
                elif event[0] == 'c': # program change; cn program
                    if running:
                        D.append( {'type':'program change', 'type hex':'c', 'omitted':1,
                                   'channel':event[1], 'program':track[i:i+2]} )
                        i += 2
                    else:
                        D.append( {'type':'program change', 'type hex':'c', 'omitted':0,
                                   'channel':track[i+1], 'program':track[i+2:i+4]} )
                        i += 4
                
                elif event[0] == '9': # note on; 9n note velocity
                    if running:
                        D.append( {'type':'note on', 'type hex':'9', 'omitted':1,
                                   'channel':event[1], 'note':track[i:i+2], 'velocity':track[i+2:i+4]} )
                        i += 4
                    else:
                        D.append( {'type':'note on', 'type hex':'9', 'omitted':0,
                                   'channel':track[i+1], 'note':track[i+2:i+4], 'velocity':track[i+4:i+6]} )
                        i += 6
                    
                elif track_first == '8': # note off; 8n note velocity
                    if running:
                        D.append( {'type':'note off', 'type hex':'8', 'omitted':1,
                                   'channel':event[1], 'note':track[i:i+2], 'velocity':track[i+2:i+4]} )
                        i += 4
                    else:
                        D.append( {'type':'note off', 'type hex':'8', 'omitted':0,
                                   'channel':track[i+1], 'note':track[i+2:i+4], 'velocity':track[i+4:i+6]} )
                        i += 6
                
                last_event = event
                state = 'delta-time'
                
        self.segs = D    
    
    def segs_to_hexstring(self): # segs is list of events, each event is a dictionary
        hexstring = ''

        for event in self.segs:
            hexstring += Track.event_to_hexstring(event)
        return hexstring

    @staticmethod
    def event_to_hexstring(event):
        if event['type'] == 'header' or event['type'] == 'chunklen' or event['type'] == 'delta time':
            hexstring = event['value']

        elif event['type'] == 'end of track':
            hexstring = event['type hex']

        elif event['type'] == 'text meta event':
            hexstring = event['type hex']*(not event['omitted']) + event['length_varlen'] + event['name']

        elif event['type'] in ['time signature', 'key signature', 'tempo', 'smpte']:
            hexstring = event['type hex'] + event['value']

        elif event['type'] == 'control change':
            hexstring = (event['type hex']+event['channel'])*(not event['omitted']) + event['controller'] + event['value']

        elif event['type'] == 'program change':
            hexstring = (event['type hex']+event['channel'])*(not event['omitted']) + event['program']

        elif event['type'] == 'note on' or event['type'] == 'note off':
            hexstring = (event['type hex']+event['channel'])*(not event['omitted']) + event['note'] + event['velocity']

        return hexstring
    
    @staticmethod
    def event_to_hexstring_discard_runningstatus(event):
        if event['type'] == 'header' or event['type'] == 'chunklen' or event['type'] == 'delta time':
            hexstring = event['value']

        elif event['type'] == 'end of track':
            hexstring = event['type hex']

        elif event['type'] == 'text meta event':
            hexstring = event['type hex'] + event['length_varlen'] + event['name']

        elif event['type'] in ['time signature', 'key signature', 'tempo', 'smpte']:
            hexstring = event['type hex'] + event['value']

        elif event['type'] == 'control change':
            hexstring = event['type hex'] + event['channel'] + event['controller'] + event['value']

        elif event['type'] == 'program change':
            hexstring = event['type hex'] + event['channel'] + event['program']

        elif event['type'] == 'note on' or event['type'] == 'note off':
            hexstring = event['type hex'] + event['channel'] + event['note'] + event['velocity']

        return hexstring
    
class Music:
    def __init__(self, name):
        self._parse_midi(name)
        self.name = name
    
    def _parse_midi(self, name):
        midi_file = f'./{name}.mid'

        with open(midi_file, 'rb') as f:
            hexdata = binascii.hexlify(f.read())
        hexdata_str = hexdata.decode("utf-8")

        MTrk = '4d54726b'
        tracks_str = []
        for i,chunk in enumerate(hexdata_str.split(MTrk)):
            if i == 0:
                header_str = chunk
            else:
                tracks_str.append (MTrk + chunk)

        track_objs = [Track(t) for t in tracks_str]
        
        self.header_str = header_str
        self.track_objs = track_objs

    def keyshift(self, shift_amount):
        track_objs = self.track_objs.copy()
        for obj in track_objs:
            for event in obj.segs:
                if event['type'] == 'note on' or event['type'] == 'note off':
                    note_hex = '0x'+event['note']
                    note_integer = int(note_hex, 16)
                    note_integer_shifted = note_integer + shift_amount
                    note_hex_shifted = hex(note_integer_shifted)[2:]
                    note_hex_shifted = note_hex_shifted.rjust(2, '0')
                    event['note'] = note_hex_shifted
        self.track_objs = track_objs
    
    def export_midi(self, name):
        recovered_tracks = [T.segs_to_hexstring() for T in self.track_objs]
        with open(f'{name}.mid', 'wb') as fout:
            fout.write( binascii.unhexlify(self.header_str + ''.join(recovered_tracks)) )

---

In [18]:
midi_file = 'beethoven.mid'

with open(midi_file, 'rb') as f:
    hexdata = binascii.hexlify(f.read())
hexdata_str = hexdata.decode("utf-8")

MTrk = '4d54726b'
tracks_str = []
for i,chunk in enumerate(hexdata_str.split(MTrk)):
    if i == 0:
        header_str = chunk
    else:
        tracks_str.append (MTrk + chunk)
print(len(tracks_str))

8


In [22]:
tempo_track = tracks_str[0]
tempo_track_obj = Track(tempo_track)
rest_hexstring = ''.join(tracks_str[1:])

In [23]:
tempo_track_obj.segs

[{'type': 'header', 'value': '4d54726b'},
 {'type': 'chunklen', 'value': '000001e4'},
 {'type': 'delta time', 'value': '00'},
 {'type': 'text meta event',
  'type hex': 'ff03',
  'omitted': 0,
  'length_varlen': '08',
  'name': '756e7469746c6564'},
 {'type': 'delta time', 'value': '00'},
 {'type': 'text meta event',
  'type hex': 'ff7f',
  'omitted': 0,
  'length_varlen': '03',
  'name': '000041'},
 {'type': 'delta time', 'value': '00'},
 {'type': 'time signature', 'type hex': 'ff5804', 'value': '04021808'},
 {'type': 'delta time', 'value': '00'},
 {'type': 'key signature', 'type hex': 'ff5902', 'value': '0000'},
 {'type': 'delta time', 'value': '00'},
 {'type': 'tempo', 'type hex': 'ff5103', 'value': '124f80'},
 {'type': 'delta time', 'value': '8d10'},
 {'type': 'tempo', 'type hex': 'ff5103', 'value': '137ab4'},
 {'type': 'delta time', 'value': '28'},
 {'type': 'tempo', 'type hex': 'ff5103', 'value': '145855'},
 {'type': 'delta time', 'value': '28'},
 {'type': 'tempo', 'type hex': 'ff

In [47]:
def convert_hex_to_arr(hexstring):
    '''
    last element of the felt array is an integer indicating the hex-length of the last felt value
    '''
    arr = []
    s = hexstring
    while( len(s)>62 ):
        felt_hex = s[0:62]
        arr.append( int(felt_hex, 16) )
        s = s[62:]
    felt_hex = s
    last_length = len(felt_hex)
    arr.append( int(felt_hex, 16) )
    arr.append(last_length)

    return arr

def generate_single(hexstring, name):
    ret = []

    ret.append( '@view')
    ret.append(f'func {name} {{')
    ret.append( '        range_check_ptr')
    ret.append( '    } () -> (')
    ret.append( '        z_len : felt,')
    ret.append( '        z : felt*')
    ret.append( '    ):')
    ret.append( '    alloc_locals\n')
    ret.append( '    let (local z) = alloc()\n')

    arr = convert_hex_to_arr(hexstring)
    for i,num in enumerate(arr):
        ret.append(f'    assert [z+{i}] = {num}')
    ret.append(f'    let z_len = {i+1}')
    ret.append('\n    return (z_len, z)')
    ret.append('end')

    for line in ret:
        print(line)

def generate_tempo_manipulable(segs, name):
    ret = []

    ret.append( '@view')
    ret.append(f'func {name} {{')
    ret.append( '        range_check_ptr')
    ret.append( '    } (')
    ret.append( '        tempo_multiplier : felt,')
    ret.append( '        tempo_divider : felt')
    ret.append( '    ) -> (')
    ret.append( '        z_len : felt,')
    ret.append( '        z : felt*')
    ret.append( '    ):')
    ret.append( '    alloc_locals\n')
    ret.append( '    let (local z) = alloc()\n')

    i=0
    for event in segs:
        if event['type'] == 'tempo':
            event_felt = int(event["type hex"], 16)
            value_felt = int(event["value"], 16)
            ret.append('')
            ret.append(f'    assert [z+{i}] = {event_felt}')
            ret.append(f'    assert [z+{i+1}] = 6') # ff 51 03
            ret.append(f'    tempvar value_ = {value_felt} * tempo_multiplier')
            ret.append(f'    let (adjusted_value, _) = unsigned_div_rem(value_, tempo_divider)')
            ret.append(f'    assert [z+{i+2}] = adjusted_value')
            ret.append(f'    assert [z+{i+3}] = 6') # tt tt tt
            i += 4
        else:
            hexstr = Track.event_to_hexstring(event)
            felt = int(hexstr, 16)
            ret.append(f'    assert [z+{i}] = {felt}')
            ret.append(f'    assert [z+{i+1}] = {len(hexstr)}') # ff 51 03 tt tt tt
            i += 2
        
    ret.append(f'    let z_len = {i}')
    ret.append('\n    return (z_len, z)')
    ret.append('end')

    for line in ret:
        print(line)

In [48]:
generate_tempo_manipulable(tempo_track_obj.segs, 'tempo')

@view
func tempo {
        range_check_ptr
    } (
        tempo_multiplier : felt,
        tempo_divider : felt
    ) -> (
        z_len : felt,
        z : felt*
    ):
    alloc_locals

    let (local z) = alloc()

    assert [z+0] = 1297379947
    assert [z+1] = 8
    assert [z+2] = 484
    assert [z+3] = 8
    assert [z+4] = 0
    assert [z+5] = 2
    assert [z+6] = 308290407136960384311518564
    assert [z+7] = 22
    assert [z+8] = 0
    assert [z+9] = 2
    assert [z+10] = 280920976261185
    assert [z+11] = 12
    assert [z+12] = 0
    assert [z+13] = 2
    assert [z+14] = 71872893331576840
    assert [z+15] = 14
    assert [z+16] = 0
    assert [z+17] = 2
    assert [z+18] = 1096709963776
    assert [z+19] = 10
    assert [z+20] = 0
    assert [z+21] = 2

    assert [z+22] = 16732419
    assert [z+23] = 6
    tempvar value_ = 1200000 * tempo_multiplier
    let (adjusted_value, _) = unsigned_div_rem(value_, tempo_divider)
    assert [z+24] = adjusted_value
    assert [z+25] = 

In [25]:
generate_single(header_str, 'header')

@view
func header {
        range_check_ptr
    } () -> (
        z_len : felt,
        z : felt*
    ):
    alloc_locals

    let (local z) = alloc()

    assert [z+0] = 1568433012465980210040715044454520
    assert [z+1] = 28
    let z_len = 2

    return (z_len, z)
end


In [26]:
generate_single(rest_hexstring, 'rest')

@view
func rest {
        range_check_ptr
    } () -> (
        z_len : felt,
        z : felt*
    ):
    alloc_locals

    let (local z) = alloc()

    assert [z+0] = 136630055383401419165459256648872088915370718355070635567016276660044325236
    assert [z+1] = 171607135751107360835153850990550848278901814907649016260995280726644490416
    assert [z+2] = 113464743677373314655073568444025063277160045892337425449062156638953152552
    assert [z+3] = 113079296978082138809168953273320639364087511599734065458855640268621764610
    assert [z+4] = 113243914718050506251437998750837951084163117612438634885412266883359251752
    assert [z+5] = 107777677702639945914294759784202598350614420395847562769232411009959143488
    assert [z+6] = 1541035458586732594269599615215932347387684275275889329446827895824680
    assert [z+7] = 113078218160200389418931272044626932058834314494626966372767322952070879258
    assert [z+8] = 113464797613325281406841269737624415586849081400865788077566515006622539560


---

In [41]:
midi_file = './test/beethoven_test.mid'

with open(midi_file, 'rb') as f:
    hexdata = binascii.hexlify(f.read())
hexdata_str = hexdata.decode("utf-8")

MTrk = '4d54726b'
tracks_str = []
for i,chunk in enumerate(hexdata_str.split(MTrk)):
    if i == 0:
        header_str = chunk
    else:
        tracks_str.append (MTrk + chunk)
print(len(tracks_str))

8


In [45]:
tracks_str[0]

'4d54726b000001e400ff0308756e7469746c656400ff7f0300004100ff58040402180800ff59020000000753008d1007caae280823552807caae8120075300a35007caae280823552807caae81200753008f7807caae280823552807caae8120075300883807a1207807f60b7808843b7808b8242808ee96280927c028075300b63007caae280823552807caae8120075300843007caae280823552807caae28075300857807a12078082355780927c07809a31a78082355814807f60b2807caae2807a1202807794428075300bb0807caae280823552807caae28075300836007caae280823552808b8242807caae87400753008648082355280927c0280a7692280753008648082355280927c0280a769228082355817008b82481700927c082680a7692280b71b0280c3500280ea60000ff2f00'

In [46]:
tempo_track

'4d54726b000001e400ff0308756e7469746c656400ff7f0300004100ff58040402180800ff5902000000ff5103124f808d10ff5103137ab428ff510314585528ff5103137ab48120ff5103124f80a350ff5103137ab428ff510314585528ff5103137ab48120ff5103124f808f78ff5103137ab428ff510314585528ff5103137ab48120ff5103124f808838ff51031312d078ff510313e71c78ff5103154a9578ff510315cc5b28ff510316547728ff510316e36028ff5103124f80b630ff5103137ab428ff510314585528ff5103137ab48120ff5103124f808430ff5103137ab428ff510314585528ff5103137ab428ff5103124f808578ff51031312d078ff510314585578ff510316e36078ff51031817c378ff51031458558148ff510313e71c28ff5103137ab428ff51031312d028ff510312af2a28ff5103124f80bb08ff5103137ab428ff510314585528ff5103137ab428ff5103124f808360ff5103137ab428ff510314585528ff510315cc5b28ff5103137ab48740ff5103124f808648ff510314585528ff510316e36028ff51031a286e28ff5103124f808648ff510314585528ff510316e36028ff51031a286e28ff51031458558170ff510315cc5b8170ff510316e3608268ff51031a286e28ff51031c9c3828ff51031e848028ff5103249f0000ff2f00'